# Wyznaczanie diagramu Voronoi poprzez triangulację Delaunay'a

In [ ]:
# import numpy as np
from bitalg.visualizer.main import Visualizer
# from random import uniform
from math import sqrt

#### Obiekty

In [ ]:
class Point:
    def __init__(self, x, y):
        self.x = x
        self.y = y
    
    def __eq__(self, other):
        return self.x == other.x and self.y == other.y
    
    def __hash__(self):
        return hash((self.x, self.y))
    
    def distance(self, p):
        return sqrt((self.x - p.x)**2 + (self.y - p.y)**2)


class Triangulation:
    def __init__(self, P):
        self.map_vertexes, self.central_point = self.get_triangulation_start(P)
        # central_point -> start_tri
    
    def get_triangulation_start(self, P):
        low_left = Point(float('inf'), float('inf'))
        up_right = Point(float('-inf'), float('-inf'))
        for p in P:
            if p.x < low_left.x:
                low_left.x = p.x
            if p.x > up_right.x:
                up_right.x = p.x
            if p.y < low_left.y:
                low_left.y = p.y
            if p.y > up_right.y:
                up_right.y = p.y
        map_vertexes = [low_left, Point(up_right.x, low_left.y), up_right, Point(low_left.x, up_right.y)]
        central_point = Point((low_left.x + up_right.x)/2, (low_left.y + up_right.y)/2 + 1e-2)
        return map_vertexes, central_point
    

class Edge:
    def __init__(self, tri1, tri2):
        self.tri1 = tri1
        self.tri2 = tri2
        self.p1, self.p2 = self.get_points()
    
    def get_points(self):
        tri1_vertexes = {self.tri1.p1, self.tri1.p2, self.tri1.p3}
        tri2_vertexes = {self.tri2.p1, self.tri2.p2, self.tri2.p3}
        points = tri1_vertexes.intersection(tri2_vertexes)
        return points[0], points[1]


def orientation(p1, p2, p3):
    return (p2.x - p1.x)*(p3.y - p2.y) - (p2.y - p1.y)*(p3.x - p2.x)
    # return < 0 -> p3 po prawej stronie prostej (p1,p2)

class Triangle:
    def __init__(self, p1, p2, p3):
        self.p1 = p1
        self.p2 = p2
        self.p3 = p3
        self.edges = self.set_edges()
        self.o, self.R = self.set_circle()

    def __contains__(self, p):
        # Test if a point is in a triangle
        a12 = orientation(self.p1, self.p2, self.p3)
        b12 = orientation(self.p1, self.p2, p)
        a23 = orientation(self.p2, self.p3, self.p1)
        b23 = orientation(self.p2, self.p3, p)
        a31 = orientation(self.p3, self.p1, self.p2)
        b31 = orientation(self.p3, self.p1, p)
        return (a12 * b12 >= 0) and (a23 * b23 >= 0) and (a31 * b31 >= 0)

    def __eq__(self, other):
        S_points = {self.p1, self.p2, self.p3}
        O_points = {other.p1, other.p2, other.p3}
        return S_points == O_points
    
    def set_circle(self):
        s1 = self.p1.x**2 + self.p1.y**2
        s2 = self.p2.x**2 + self.p2.y**2
        s3 = self.p3.x**2 + self.p3.y**2
        x13 = self.p1.x - self.p3.x
        x32 = self.p3.x - self.p2.x
        x21 = self.p2.x - self.p1.x
        y12 = self.p1.y - self.p2.y
        y23 = self.p2.y - self.p3.y
        y31 = self.p3.y - self.p1.y
        f = 2*(self.p1.x*y23 + self.p2.x*y31 + self.p3.x*y12)
        # Środek okręgu opisanego na trójkącie ma środek:
        x0 = (s1*y23 + s2*y31 * s3*y12)/f
        y0 = (s1*x32 + s2*x13 * s3*x21)/f
        # Promień okręgu opisanego na trójkącie wynosi:
        R = sqrt((x21**2 + y12**2) * (x13**2 + y31**2) * (x32**2 + y23**2))/abs(f)
        return Point(x0, y0), R

    def in_circle(self, p):
        if self.o.distance(p) < self.o.distance(self.p1):
            return True
        else:
            return False
        
    def set_edges(self):
        edges = []
        e1 = Edge(self, )

#### Funkcje pomocnicze funkcji delauney

In [ ]:
def get_points(Points):
    P = []
    for point in Points:
        P.append(Point(point[0], point[1]))
    return P

def find_containing(p):
    pass

def get_adjacent_tri(triangle):
    nerby_triangles = []
    for edge in triangle.edges:
        if edge.tri1 != triangle:
            nerby_triangles.append(edge.tri1)
        if edge.tri2 != triangle:
            nerby_triangles.append(edge.tri2)
    return nerby_triangles

def adjust_triangulation(tri_to_remove, p):
    edges_to_check = []
    # for tri_to_remove:
    #     pass
        

#### Główna funkcja delauney

In [ ]:
def delauney(Points):
    P = get_points(Points)
    T = Triangulation(P)
    start_triangulation1 = Triangle(T.map_vertexes[0], T.map_vertexes[1], T.map_vertexes[2])
    start_triangulation2 = Triangle(T.map_vertexes[2], T.map_vertexes[3], T.map_vertexes[0])

    for p in P:

        containing_tri = find_containing(p)
        tri_to_remove = []
        tri_visited = []
        stack = [containing_tri]

        while len(stack) > 0:
            curr_tri = stack.pop()
            tri_visited.append(curr_tri)

            if curr_tri.in_circle(p):
                tri_to_remove.append(curr_tri)
                tri_adjacent = get_adjacent_tri(curr_tri)

                for triangle in tri_adjacent:
                    if triangle not in tri_visited and triangle not in stack:
                        stack.append(triangle)
            
            adjust_triangulation(tri_to_remove, p)
        
        

#### Dane testowe - delauney